# Download the Dataset

In [ ]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="jA7vYeRcSwu0IZIPTibv")
project = rf.workspace("xyz-rtzwg").project("new-f6gjk")
dataset = project.version(2).download("yolov8")


# Train the Dataset

In [ ]:
!yolo train data='new-2/data.yaml' model='yolov8s.pt' epochs=100 imgsz=64

# Predict the model on Test images

In [ ]:
!yolo predict model='runs/detect/train/weights/best.pt' source='new-2/test/images/' conf=0.5

# Test the model on webcam

In [ ]:
from ultralytics import YOLO
import cv2
import cvzone
import math

cap=cv2.VideoCapture(0)
model=YOLO('runs/detect/train/weights/best.pt') # Change the path to your final-weights.pt file to check on our model
classnames=['ball', 'bottle', 'mug','nutella','paperbag', 'shoes','ketchup','cup']
while True:
    ret, frame=cap.read()
    # print(frame.shape)
    # frame=frame[0:300,:]
    results=model(frame,stream=True)
    for r in results:
        boxes=r.boxes
        for box in boxes:
            x1,y1,x2,y2=box.xyxy[0]
            x1,y1,x2,y2=int(x1),int(y1),int(x2),int(y2)
            w=x2-x1
            h=y2-y1

            bbox=int(x1),int(y1),int(w),int(h)
            confidence=math.ceil((box.conf[0])*100)/100
            
            cls=int(box.cls[0])
            if confidence>0.50:
                cvzone.cornerRect(frame,bbox)
                cvzone.putTextRect(frame,f'{classnames[cls]}' f'{confidence}',(max(0,x1),max(30,y1)))
    
    cv2.imshow("Front cam",frame)
    if cv2.waitKey(1)== ord('q'):
        break

cap.release()
# cv2.destroyAllWindows

# Test the model on Rpi camera

In [ ]:
from ultralytics import YOLO
import cv2
import cvzone
import math
import io
import socket
import struct
from PIL import Image
import numpy

model=YOLO('runs/detect/train/weights/best.pt')
print('Model Initialized!!!')

server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
server_socket.bind(('0.0.0.0', 8000))  
server_socket.listen(0)
print("Listening")
classnames=['ball', 'bottle', 'mug','nutella','paperbag', 'shoes','ketchup','cup']
client_socket, client_address = server_socket.accept()
print("Connected to client:", client_address)
response = '0'
connection = client_socket.makefile('rb')
try:
    img = None
    while True:
        print('s')
        image_len = struct.unpack('<L', connection.read(struct.calcsize('<L')))[0]
        if not image_len:
            break
        image_stream = io.BytesIO()
        image_stream.write(connection.read(image_len))
        image_stream.seek(0)
        image = Image.open(image_stream)
        im = cv2.cvtColor(numpy.array(image), cv2.COLOR_RGB2BGR)
        frame=im
        results=model(frame,stream=True)
        print('result')
        response = '0'
        for r in results:
            boxes=r.boxes
            for box in boxes:
                x1,y1,x2,y2=box.xyxy[0]
                x1,y1,x2,y2=int(x1),int(y1),int(x2),int(y2)
                w=x2-x1
                h=y2-y1
                bbox=int(x1),int(y1),int(w),int(h)
                confidence=math.ceil((box.conf[0])*100)/100
                
                cls=int(box.cls[0])
                if confidence>0.6:
                    cvzone.cornerRect(frame,bbox)
                    cvzone.putTextRect(frame,f'{classnames[cls]}' f'{confidence}',(max(0,x1),max(30,y1)))
                    response = '1'
                
        client_socket.send(response.encode())
        response = '0'
        cv2.imshow("Front Cam",frame)
        if cv2.waitKey(1)== ord('q'):
            break

    cv2.destroyAllWindows()
except: 
    print('Error Occured')
finally:
    connection.close()
    server_socket.close()

cv2.destroyAllWindows